In [1]:
import pandas as pd
import numpy as np
import random
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
import timeit

pd.set_option('display.max_rows', 60000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
pd.options.display.float_format = '{:,.2f}'.format

import ipywidgets
from bokeh.io import push_notebook
from bokeh.models import Range1d
from bokeh.plotting import figure
from bokeh.models import HoverTool
# standard bokeh imports
from bokeh.io import output_notebook, show, reset_output
output_notebook()

Loading BokehJS ...

In [2]:
from pymongo import MongoClient
client = MongoClient()

#function to transform collections into df 
def extract_(set_):
    db = client['hdb']
    collection =db[set_]
    df_ = pd.DataFrame(list(collection.find()))
    df_ = df_.drop(['_id'], axis=1)
    return(df_)

set_hdb = ['set_1','set_2','set_3','set_4'] #names of collections in hdb 
combine_df = [extract_(x) for x in set_hdb] #sending each collection to be processed by the extract_ function
df = pd.concat(combine_df,axis=0).reset_index(drop=True) #putting them together in one df 

In [3]:
qtr_map = {'01':'Q1','02':'Q1','03':'Q1',
           '04':'Q2','05':'Q2','06':'Q2',
           '07':'Q3','08':'Q3','09':'Q3',
           '10':'Q4','11':'Q4','12':'Q4'}

mth_map = {'01':'Jan','02':'Feb','03':'Mar',
           '04':'Apr','05':'May','06':'Jun',
           '07':'Jul','08':'Aug','09':'Sep',
           '10':'Oct','11':'Nov','12':'Dec'}

df['year'] = df['month'].map(lambda x: x[0:4])
df['mth_'] = df['month'].map(lambda x: x[5:])
df['mth_'] = df['mth_'].map(mth_map)
df['qtr_'] = df['month'].map(lambda x: x[5:]).map(qtr_map)
df['year_qtr'] = df['year']+"-"+df['qtr_']
df['resale_p'] = df['resale_price']/1000
df['flat_type'] = df['flat_type'].str.replace('MULTI-GENERATION','EXECUTIVE')

#mapping as consulted: http://www.propertyhub.com.sg/singapore-district-guide.html
location_map ={'ANG MO KIO':'Central', 'BEDOK':'East', 'BISHAN':'Central','BUKIT BATOK':'West','BUKIT MERAH':'Central',
               'BUKIT PANJANG':'West','BUKIT TIMAH':'Central','CENTRAL AREA':'Central','CHOA CHU KANG':'West',
               'CLEMENTI':'West','GEYLANG':'Central','HOUGANG':'North-East','JURONG EAST':'West','JURONG WEST':'West',
              'KALLANG/WHAMPOA':'Central', 'MARINE PARADE':'East', 'PASIR RIS':'East', 'PUNGGOL':'North-East',
              'QUEENSTOWN':'Central', 'SEMBAWANG':'North', 'SENGKANG':'North-East', 
              'SERANGOON':'North-East', 'TAMPINES':'East','TOA PAYOH': 'Central','WOODLANDS':'North', 'YISHUN':'North'}
locations = list(df['town'])
df['zones'] = [location_map[loc] for loc in locations] #mapped to zones 

## EDA 

In [4]:
df.info() #summary of the combine dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549130 entries, 0 to 549129
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                549130 non-null  object 
 1   town                 549130 non-null  object 
 2   flat_type            549130 non-null  object 
 3   block                549130 non-null  object 
 4   street_name          549130 non-null  object 
 5   storey_range         549130 non-null  object 
 6   floor_area_sqm       549130 non-null  float64
 7   flat_model           549130 non-null  object 
 8   lease_commence_date  549130 non-null  int64  
 9   resale_price         549130 non-null  float64
 10  year                 549130 non-null  object 
 11  mth_                 549130 non-null  object 
 12  qtr_                 549130 non-null  object 
 13  year_qtr             549130 non-null  object 
 14  resale_p             549130 non-null  float64
 15  zones            

In [5]:
print (df.shape)  #this is the shape of the dataset 

(549130, 16)


In [15]:
df.columns #these are the columns in the dataset

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'year', 'mth_', 'qtr_', 'year_qtr', 'resale_p', 'zones'],
      dtype='object')

In [16]:
df.isnull().sum() #ensure no nans in the dataset

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
year                   0
mth_                   0
qtr_                   0
year_qtr               0
resale_p               0
zones                  0
dtype: int64

In [17]:
df.describe() #a descriptive statistical summary of the dataset

,floor_area_sqm,lease_commence_date,resale_price,resale_p
count,"549,130.00","549,130.00","549,130.00","549,130.00"
mean,96.77,"1,989.38","335,570.29",335.57
std,25.24,10.35,"145,427.92",145.43
min,28.00,"1,966.00","28,000.00",28.00
25%,73.00,"1,982.00","228,000.00",228.00
50%,98.00,"1,988.00","315,500.00",315.50
75%,114.00,"1,997.00","416,000.00",416.00
max,297.00,"2,019.00","1,258,000.00","1,258.00"


In [6]:
len(df.year.unique()) #21 years worth of data! 

22

In [7]:
df.sample(5,random_state = 42)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,year,mth_,qtr_,year_qtr,resale_p,zones
326609,2010-06,TOA PAYOH,3 ROOM,59,LOR 5 TOA PAYOH,01 TO 03,61.00,Standard,1973,"250,000.00",2010,Jun,Q2,2010-Q2,250.00,Central
475640,2017-10,TOA PAYOH,3 ROOM,91,LOR 3 TOA PAYOH,04 TO 06,68.00,Improved,1970,"225,000.00",2017,Oct,Q4,2017-Q4,225.00,Central
408462,2014-03,JURONG WEST,3 ROOM,421,JURONG WEST ST 42,07 TO 09,64.00,Simplified,1985,"306,000.00",2014,Mar,Q1,2014-Q1,306.00,West
494737,2018-09,JURONG WEST,5 ROOM,622,JURONG WEST ST 61,10 TO 12,110.00,Improved,2001,"425,000.00",2018,Sep,Q3,2018-Q3,425.00,West
245902,2007-10,HOUGANG,4 ROOM,506,HOUGANG AVE 8,01 TO 03,84.00,Simplified,1987,"180,000.00",2007,Oct,Q4,2007-Q4,180.00,North-East


In [8]:
df['flat_type'].unique()

array(['3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '2 ROOM', '1 ROOM'],
      dtype=object)

In [9]:
df.town.unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'QUEENSTOWN',
       'SENGKANG', 'SERANGOON', 'TAMPINES', 'TOA PAYOH', 'WOODLANDS',
       'YISHUN', 'SEMBAWANG', 'PUNGGOL'], dtype=object)

In [10]:
df['flat_type'].unique()

array(['3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '2 ROOM', '1 ROOM'],
      dtype=object)

In [11]:
pd.crosstab(df['town'],df['flat_type']) # a high level view of the total no. of transactions 

flat_type,1 ROOM,2 ROOM,3 ROOM,4 ROOM,5 ROOM,EXECUTIVE
town,,,,,,
ANG MO KIO,0,533,17527,6813,2951,324
BEDOK,0,474,16234,10832,5647,1593
BISHAN,0,0,1626,5737,2987,1076
BUKIT BATOK,0,0,9915,9451,2942,2021
BUKIT MERAH,449,757,8760,6214,4091,5
BUKIT PANJANG,0,52,2517,9082,5535,1879
BUKIT TIMAH,0,0,297,503,340,233
CENTRAL AREA,0,224,2211,1244,254,2
CHOA CHU KANG,0,39,1276,13094,7928,2784


### Price Compare App

In [12]:
town_list = list(df.town.unique())
year_list = list(df.year.unique())
year_list.sort(reverse=True)
mth_list = list(df.mth_.unique())
zone_list = list(df.zones.unique())
flat_list = list(df.flat_type.unique())
flat_list.sort()
view_options = ['median price','mean price','no. of transactions' ]

In [13]:
# widget
#1. town_1 type
drop_down1 = ipywidgets.Dropdown(options=town_list,value=town_list[0],
                                description='Town 1:',disabled=False)
#2. town_2 type
drop_down2 = ipywidgets.Dropdown(options=town_list,value=town_list[1],
                                description='Town 2:',disabled=False)

#3. flat type
drop_down3 = ipywidgets.Dropdown(options=flat_list,value=flat_list[-1],
                                description='Flat Type:',disabled=False)
#4.year 
drop_down4 = ipywidgets.Dropdown(options=year_list,value=year_list[0],
                                description='Year:',disabled=False)
#5.month 
drop_down5 = ipywidgets.Dropdown(options=mth_list,value=mth_list[0],
                                description='Month:',disabled=False)

#6. view of transactions
drop_down6 = ipywidgets.Dropdown(options=view_options,value=view_options[0],
                                description='View By:', disabled=False)

In [14]:
def interact_ (town_1,town_2, flat_type,year,mth, options): 

    # data -- initial set up
    x_bar_data_ipyw = [town_1,town_2]
    y_bar_data_ipyw = [df[(df.year==year)&(df.mth_==mth)&(df.town == town_1)&(df.flat_type == flat_type)]['resale_p'].median(), 
                   df[(df.year==year)&(df.mth_==mth)&(df.town == town_2)&(df.flat_type == flat_type)]['resale_p'].median()] 
    # figure and plot
    bar_chart_interactive = figure(x_range=x_bar_data_ipyw, plot_height=300, title='HDB Price Comparison')
    bar_ipyw = bar_chart_interactive.vbar(x_bar_data_ipyw, top=y_bar_data_ipyw, color='blue', width=0.5)
    #bar_chart_interactive.y_range=Range1d(0, 1200) #note shifted to the if-else statement below
    bar_chart_interactive.add_tools(HoverTool()) #to produce the tooltip 
    
    if options == 'median price':
        fitting = [df[(df.year==year)&(df.mth_==mth)&(df.town == town_1)&(df.flat_type == flat_type)]['resale_p'].median(), 
                   df[(df.year==year)&(df.mth_==mth)&(df.town == town_2)&(df.flat_type == flat_type)]['resale_p'].median()]
        bar_chart_interactive.y_range=Range1d(0, 1200)
    elif options == 'mean price': 
        fitting = [df[(df.year==year)&(df.mth_==mth)&(df.town == town_1)&(df.flat_type == flat_type)]['resale_p'].mean(), 
                   df[(df.year==year)&(df.mth_==mth)&(df.town == town_2)&(df.flat_type == flat_type)]['resale_p'].mean()]
        bar_chart_interactive.y_range=Range1d(0, 1200)
    elif options == 'no. of transactions':
        fitting = [df[(df.year==year)&(df.mth_==mth)&(df.town == town_1)&(df.flat_type == flat_type)]['resale_p'].count(), 
                   df[(df.year==year)&(df.mth_==mth)&(df.town == town_2)&(df.flat_type == flat_type)]['resale_p'].count()]
        bar_chart_interactive.y_range=Range1d(0, 300)
    
    bar_ipyw.data_source.data['top'] = fitting
    show(bar_chart_interactive, notebook_handle=True)
    push_notebook()
    
ipywidgets.interact(interact_, town_1 = drop_down1, town_2= drop_down2, flat_type=drop_down3, 
                    year=drop_down4, mth=drop_down5, options=drop_down6);

interactive(children=(Dropdown(description='Town 1:', options=('ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK',…